# Training an Iris classifier using Keras
In this notebook, we'll show how to train a classifier trained on the [iris data set](https://archive.ics.uci.edu/ml/datasets/iris) using Keras.

## Install Dependencies
First, we'll install our dependencies:

In [0]:
pip install keras==2.3.* scikit-learn==0.21.* keras2onnx==1.5.* boto3==1.*

## Load the data
We can use scikit-learn to load the Iris dataset:

In [0]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

iris = load_iris()
X, y = iris.data, np_utils.to_categorical(iris.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

## Train the model
We'll use Keras to define the neural net and train the model:

In [0]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential(name="iris")
model.add(Dense(30, input_dim=4, activation="relu", name="input"))
model.add(Dense(3, activation="softmax", name="last"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=300)

scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))  # Accuracy should be > 90%

## Export the model
Now we can export the model in the ONNX format:

In [0]:
import keras2onnx

onnx_model = keras2onnx.convert_keras(model)

with open("keras.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Upload the model to AWS

Cortex loads models from AWS, so we need to upload the exported model.

Set these variables to configure your AWS credentials and model upload path:

In [0]:
AWS_ACCESS_KEY_ID = "" #@param {type:"string"}
AWS_SECRET_ACCESS_KEY = "" #@param {type:"string"}
S3_UPLOAD_PATH = "s3://my-bucket/iris-classifier/keras.onnx" #@param {type:"string"}

import sys
import re

if AWS_ACCESS_KEY_ID == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_ACCESS_KEY_ID"), file=sys.stderr)

elif AWS_SECRET_ACCESS_KEY == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_SECRET_ACCESS_KEY"), file=sys.stderr)

else:
    try:
        bucket = re.search("s3://(.+?)/", S3_UPLOAD_PATH).group(1)
        key = re.search("s3://.+?/(.+)", S3_UPLOAD_PATH).group(1)
    except:
        print("\033[91m{}\033[00m".format("ERROR: Invalid s3 path (should be of the form s3://my-bucket/path/to/file)"), file=sys.stderr)

Upload the model to S3:

In [0]:
import boto3

s3 = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
print("Uploading {} ...".format(S3_UPLOAD_PATH), end = '')
s3.upload_file("keras.onnx", bucket, key)
print(" ✓")

<!-- CORTEX_VERSION_MINOR -->
That's it! See the [example on GitHub](https://github.com/cortexlabs/cortex/tree/0.9/examples/iris-classifier) for how to deploy the model as an API.